# Sound Magician

Thời lượng ước tính: **120** phút

## Giới thiệu lab
Nâng cao! Có thể bạn đã từng xem một bộ phim hoặc chương trình truyền hình mà các điệp viên siêu hạng sử dụng một số kỹ thuật khó tin để nâng cao hình ảnh pixelated hoặc khôi phục một số dữ liệu bị mất. Trong lab này, bạn sẽ thực hiện điều tương tự — lần này mục tiêu của bạn là sử dụng hồi quy tuyến tính để khôi phục hoặc 'điền vào' phần bị xóa hoàn toàn của một tệp âm thanh!

Để hoàn thành lab này, bạn sẽ sử dụng FSDD (Free-Spoken-Digits-Dataset), một tập dữ liệu âm thanh do Zohar Jackson tổng hợp lại khi anh ấy nhận thấy không có nhiều âm thanh được làm sạch (không có khoảng trống, độ dài tương đương, cùng bitrate, cùng tỷ lệ mẫu trên giây, ...) thư viện âm thanh sẵn sàng cho machine learning.

In [1]:
import numpy as np
import pandas as pd

from sklearn.utils.validation import check_random_state
import scipy.io.wavfile as wavfile

### Về Âm thanh

Mẫu là các quan sát. Mỗi tệp âm thanh sẽ là một mẫu duy nhất trong tập dữ liệu.

Tìm thêm thông tin về [Mẫu âm thanh tại đây] (https://en.wikipedia.org/wiki/Sampling_(signal_processing)).

Mỗi tệp .wav thực sự chỉ là một loạt các mẫu số, được lấy mẫu từ tín hiệu analog. Sampling (Lấy mẫu) là một kiểu rời rạc. Khi chúng ta đề cập đến các 'sample (mẫu)', tức là các quan sát. Khi đề cập đến 'audio sample', chúng ta muốn nói đến "feature (thuộc tính)" thực sự của tệp âm thanh.

Mục tiêu của lab này là sử dụng hồi quy tuyến tính, đa mục tiêu để tạo ra bằng phép ngoại suy phần bị thiếu của tệp âm thanh thử nghiệm.

Mỗi một thuộc tính audio_sample sẽ là đầu ra của một phương trình, là một hàm của phần được cung cấp của audio_samples:

    missing_samples = f(provided_samples)

Bạn có thể thử với lượng âm thanh bạn muốn cắt và để máy tính tạo ra bằng tham số Provided_Portion.

Hãy sử dụng cái này. Đây là lượng tệp âm thanh sẽ được cung cấp, tính bằng phần trăm. Phần trăm còn lại của tệp sẽ được tạo thông qua phép ngoại suy tuyến tính.

In [2]:
Provided_Portion = 0.25

## 1. Chuẩn bị dữ liệu

Bắt đầu bằng cách tạo một list Python thông thường là `zero`:

In [3]:
# Nhập code của bạn ở đây
import glob
data_wav = glob.glob('data/*.wav')
len(data_wav)

50

In [25]:
zero = []
for wav_file in data_wav:
    sample_rate, data = wavfile.read(wav_file, mmap=False)
    #print('Sample Rate: {}, Data: {}'.format(samplerate,data))
    zero.append(data)

Lặp qua tập dữ liệu và tải lên tất cả 50 tệp `0_jackson*.wav` bằng phương thức `wavfile.read()`: https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.io.wavfile.read.html từ thư mục [data](https://drive.google.com/drive/folders/1UDAvuRfXzcOcpI84neQAmAeffIrJKKBi?usp=sharing).

Hãy cẩn thận! `.read()` trả về một tuple và bạn chỉ quan tâm đến dữ liệu âm thanh chứ không phải sample_rate tại thời điểm này. Bên trong vòng lặp for, chỉ cần nối dữ liệu âm thanh đã tải vào list Python `zero`:

In [5]:
# Nhập code của bạn ở đây
zero

[array([343, 404, 454, ..., 420, 377, 345], dtype=int16),
 array([-311,  -91, -140, ...,  378,  357,  333], dtype=int16),
 array([ 320,  372,  421, ..., -234, -264, -302], dtype=int16),
 array([-369, -431, -475, ...,  301,  324,  304], dtype=int16),
 array([-361, -226, -238, ..., -286, -311, -343], dtype=int16),
 array([ 301,  394,  507, ..., -423, -401, -329], dtype=int16),
 array([ 323,  338,  357, ..., -246, -280, -301], dtype=int16),
 array([-316, -377, -435, ..., -325, -311, -301], dtype=int16),
 array([ 322,  395,  470, ..., -344, -343, -319], dtype=int16),
 array([-417,  152,  168, ...,  312,  316,  309], dtype=int16),
 array([355, 425, 481, ..., 398, 372, 332], dtype=int16),
 array([ 364,  420,  469, ..., -326, -334, -351], dtype=int16),
 array([-302, -312, -103, ..., -338, -333, -348], dtype=int16),
 array([-307, -334, -336, ...,  354,  328,  312], dtype=int16),
 array([ 305,  365,  419, ..., -313, -342, -346], dtype=int16),
 array([ 331,  404,  490, ..., -297, -307, -318], dt

Hãy dành một chút thời gian để chuyển 0 thành một khung dữ liệu (DataFrame). Khi bạn làm như vậy, hãy đặt `dtype` thành `np.int16`, vì các tệp âm thanh đầu vào là 16 bit cho mỗi mẫu. Nếu bạn không biết cách thực hiện việc này, hãy đọc tài liệu tại đây: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html

Tiếc là những đoạn âm thanh này không được chuẩn hóa độ dài nên chúng ta sẽ phải chia nhỏ chúng để tất cả có cùng độ dài. Vì Pandas sẽ chèn các NAN tại bất kỳ vị trí nào để tạo thành mảng [n_observed_samples, n_audio_samples] hình chữ nhật hoàn hảo nên hãy thực hiện một `dropna` trên trục Y ở đây. Sau đó, chuyển đổi 1 trở lại thành NDArray bằng `yourarrayname.values`:

In [6]:
# Nhập code của bạn ở đây
df = pd.DataFrame(zero,dtype=np.int16)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,6263,6264,6265,6266,6267,6268,6269,6270,6271,6272
0,343,404,454,477,477,462,425,393,364,311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-311,-91,-140,-182,-271,-68,-235,-359,-129,-198,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,320,372,421,468,514,544,564,576,581,565,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-369,-431,-475,-543,-571,-557,-528,-455,-394,-305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-361,-226,-238,-478,-425,-395,-663,-879,-726,-997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.dropna(axis=1,inplace=True)

In [8]:
df.isnull().sum()

0       0
1       0
2       0
3       0
4       0
       ..
4082    0
4083    0
4084    0
4085    0
4086    0
Length: 4087, dtype: int64

In [17]:
df = df.values
df

array([[ 343,  404,  454, ..., 2000, 3001, 3651],
       [-311,  -91, -140, ..., -422, -462, -460],
       [ 320,  372,  421, ..., 1433, 1616, 1427],
       ...,
       [-312, -335, -338, ...,  644,  815,  580],
       [-302, -337, -371, ..., 1105,  559,  477],
       [-439, -572, -656, ..., 1026, 2322, 2618]], dtype=int16)

In [18]:
print (type(zero) )

<class 'list'>


Điều quan trọng là phải biết  (bao nhiêu mẫu audio_samples) độ dài dữ liệu hiện có.

`zero` hiện có shape giống như` [n_samples, n_audio_samples] `, vì vậy hãy lấy số lượng` n_audio_samples` và lưu trữ nó trong biến `n_audio_samples`:

In [19]:
# Nhập code của bạn ở đây
n_audio_samples = df.shape[1]
print (n_audio_samples)

4087


## 2. Huấn luyện mô hình

Tạo mô hình hồi quy tuyến tính của bạn tại đây và lưu trữ nó trong biến `model`. Đừng vội huấn luyện hoặc làm bất cứ điều gì khác với nó:

In [20]:
# Nhập code của bạn ở đây
from sklearn import linear_model
model = linear_model.LinearRegression()

Có 50 bản ghi mỗi clip. Bạn chỉ muốn lấy ngẫu nhiên một trong số chúng ra và cái đó sẽ KHÔNG được sử dụng trong quá trình huấn luyện mô hình. Nói cách khác, tệp mà chúng ta sẽ kiểm tra/chấm điểm sẽ là một mẫu không nhìn thấy, độc lập với phần còn lại của training set:

In [22]:
# Hãy để nguyên dòng này cho đến khi bạn đã submit bài Lab của mình:
rng = check_random_state(7)

random_idx = rng.randint(df.shape[0])
test  = df[random_idx]
train = np.delete(df, [random_idx], axis=0)

In ra shape của `train` và `test`.

`train` sẽ có dạng:`[n_samples, n_audio_samples]`, trong đó `n_audio_samples` là 'features' (thuộc tính) của tệp âm thanh

`test` sẽ có dạng `[n_audio_features]`, vì nó là mẫu duy nhất (tệp âm thanh, ví dụ: quan sát).

In [23]:
# Nhập code của bạn ở đây
print ("Shapes of train and test:", train.shape, test.shape)

Shapes of train and test: (49, 4087) (4087,)


Dữ liệu thử nghiệm sẽ có hai phần, `X_test` và` y_test`.

`X_test` là phần đầu tiên của tệp âm thanh thử nghiệm mà chúng ta sẽ cung cấp cho máy tính làm input.

`y_test`,"nhãn" sẽ là phần còn lại của tệp âm thanh. Như vậy, máy tính sẽ sử dụng hồi quy tuyến tính để lấy ra phần bị thiếu của tệp âm thanh dựa trên dữ liệu huấn luyện mà nó nhận được!

Hãy lưu clip `test` ban đầu, clip mà bạn sắp xóa một nửa vào thư mục hiện tại để bạn có thể so sánh nó với clip 'đã vá' sau khi tạo. Bạn hẳn đã có `sample_rate` khi tải lên các tệp .wav: 

In [26]:
wavfile.write('Original Test Clip.wav', sample_rate, test)

Chuẩn bị dữ liệu TEST bằng cách tạo lát `X_test`. Nó phải có các thuộc tính mẫu âm thanh `Provided_Portion` * `n_audio_samples`, được lấy từ tệp âm thanh thử nghiệm, hiện được lưu trữ trong biến `test`. Nói cách khác, lấy các thuộc tính âm thanh `Provided_Portion` * `n_audio_samples` ĐẦU TIÊN từ `test` và lưu trữ nó trong` X_test`. Điều này sẽ được thực hiện bằng cách sử dụng lập chỉ mục:

In [29]:
# Nhập code của bạn ở đây
X_test = test[:int(Provided_Portion*n_audio_samples)]
X_test

array([-312, -335, -338, ..., 3429, 5376, 6313], dtype=int16)

Nếu các thuộc tính `Provided_Portion` * `n_audio_samples` đầu tiên được lưu trữ trong `X_test`, thì chúng ta cũng cần lấy các thuộc tính âm thanh _còn lại_ và lưu trữ chúng trong` y_test`. Với các thuộc tính còn lại được lưu trữ trong đó, chúng ta sẽ có thể R ^ 2 "chấm điểm" thuật toán đã làm tốt như thế nào trong việc hoàn thành tệp âm thanh.

In [30]:
# Nhập code của bạn ở đây
y_test = test[int(Provided_Portion*n_audio_samples):]
y_test

array([7931, 8749, 7650, ...,  644,  815,  580], dtype=int16)

Lặp lại quy trình tương tự cho `X_train`, `y_train`. Sự khác biệt duy nhất là:

1. Bạn sẽ nhận được dữ liệu âm thanh từ `train` thay vì từ `test`
2. Bạn có nhớ shape của `train` mà bạn đã in ra trước đó không? Bạn muốn thực hiện việc cắt này nhưng đối với TẤT CẢ các mẫu (quan sát). Đối với mỗ quan sát, bạn muốn chia các thuộc tính âm thanh `Provided_Portion` * `n_audio_samples` đầu tiên thành `X_train`, và phần còn lại chuyển vào `y_train`. Có thể thực hiện tất cả những điều này bằng lập chỉ mục thông thường ở 2 dòng code:

In [33]:
# Nhập code của bạn ở đây
X_train = train[:, :int(Provided_Portion*n_audio_samples)]
y_train = train[:, int(Provided_Portion*n_audio_samples):]

SciKit-Learn sẽ 'tức giận' nếu bạn không cung cấp dữ liệu huấn luyện dưới dạng khung dữ liệu 2D có shape `[n_samples, n_features]`.

Vì vậy, nếu bạn chỉ có một MẪU, chẳng hạn như trường hợp của chúng ta với `X_test` và `y_test` thì bằng cách gọi `.reshape(1, -1)`, bạn có thể biến `[n_features]` thành `[1, n_features] `để đáp ứng SciKit-Learn.

Mặt khác, nếu bạn chỉ có một THUỘC TÍNH, bạn có thể gọi `.reshape (-1, 1)` trên dữ liệu của mình để biến `[n_samples]` thành `[n_samples, 1]`.

Định hình lại X_test và y_test thành [1, n_features]:

In [34]:
# Nhập code của bạn ở đây
X_test = X_test.reshape(1, -1)
y_test = y_test.reshape(1, -1)

Điều chỉnh mô hình của bạn bằng cách sử dụng dữ liệu huấn luyện và nhãn:

In [35]:
# Nhập code của bạn ở đây
model.fit(X_train, y_train)

LinearRegression()

## 3. Đánh giá mô hình

Sử dụng mô hình để dự đoán `label` của `X_test`. Lưu trữ dự đoán kết quả trong biến `y_test_prediction`:

In [36]:
# Nhập code của bạn ở đây
y_test_prediction = model.predict(X_test)

SciKit-Learn sẽ sử dụng float64 để tạo các dự đoán của bạn, vì vậy hãy đưa các giá trị đó trở lại int16, đó là những gì tệp .wav cần:

In [37]:
y_test_prediction = y_test_prediction.astype(dtype=np.int16)

Chấm điểm dự đoán của bạn sẽ hoạt động tốt như thế nào bằng cách truyền dữ liệu và nhãn thử nghiệm `y_test`:

In [49]:
# Nhập code của bạn ở đây
from sklearn.metrics import r2_score
score = r2_score(y_test_prediction,y_test)

/Users/chinguyen/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/metrics/_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [52]:
print("Extrapolation R^2 Score: ", score)

Extrapolation R^2 Score:  nan


Hãy lấy phần `Provided_Portion` đầu tiên của clip thử nghiệm, phần mà bạn đã đưa vào mô hình hồi quy tuyến tính. Sau đó, kết hợp điều đó với thứ mà mô hình dự đoán đã tạo cho bạn và sau đó lưu đoạn âm thanh đã hoàn thành:

In [40]:
completed_clip = np.hstack((X_test, y_test_prediction))
wavfile.write('Extrapolated Clip.wav', sample_rate, completed_clip[0])